# Geometric square
https://geodata-frisco.hub.arcgis.com/datasets/frisco::citylimits/explore

In [1]:


import pandas as pd
from pyproj import Proj, transform

data = pd.read_csv('Frisco_1km.csv')
inProj = Proj(init='epsg:3857')
outProj = Proj(init='epsg:4326')
def f(x):
    return transform(inProj, outProj, x['X'], x['Y'])

coordinates = data.apply(lambda x: f(x), result_type='expand', axis=1)
data = pd.concat([data, coordinates], axis=1)


c:\Users\jacob\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\jacob\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\jacob\AppData\Local\Temp/ipykernel_26896/792836779.py:8: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/go

In [2]:
data.to_csv('./convertedlocs.csv',index=None)

In [2]:
import requests
import json
import time
import pickle
import pandas as pd

+ Replace API_KEY
+ This endpoint is for business search

In [34]:
pos1=data.iloc[0][0]
pos2=data.iloc[0][1]

In [3]:
API_KEY = 'l6CQDw10V0hgHFzX_u14o4ljYf8ok6hEp6o3uJQRfPB805dh24sAKipHyPCE-8HsMoYeRuLbiXnNnreoS_1f101cLDBLmMnHzbFGict5L036QHKw1dzqfMYGCTtXY3Yx'
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API_KEY}

### Junk TESTS

In [48]:
PARAMETERS = {'term': 'restaurant',
                  'offset': 0, # this offset is to prevent duplicates
                  'limit': 50, 
                  'latitude':float(pos2),
                  'longitude':float(pos1),
                  'radius':1000}

In [49]:
response = requests.get(url = ENDPOINT, params= PARAMETERS, headers = HEADERS)

200

## Functions

In [4]:
def build_PARAMS(lat,long,radius,offset):
    params = {'term': 'restaurant',
                  'offset': offset, # this offset is to prevent duplicates
                  'limit': 50, 
                  'latitude':float(lat),
                  'longitude':float(long),
                  'radius':radius}
    return(params)

def handle_good_return(response,output):
    business_data = response.json()
    output['businesses'] += business_data['businesses']

def handle_bad_response(lat,long,offset_val,output):
    retry_flag=True
    for i in range(0,3):
        if retry_flag:
            time.sleep(10)
            PARAMETERS=build_PARAMS(lat,long,1000,offset_val)
            retry_request=requests.get(url = ENDPOINT, params= PARAMETERS, 
                headers = HEADERS)
            code=retry_request.status_code
            if code==200:
                retry_flag=False
                output=handle_good_return(retry_request,output)
                break
            else:
                print(f'Attempt number {i} failed will retry')
                remaining=int(retry_request.headers['ratelimit-remaining'])
                if remaining <= 10:
                    print(f'Maybe hitting daily limit will sleep until then and retry')
                    handle_sleep(retry_request)
                pass
    if retry_flag:
        raise Exception (f'Something is broken here is lat\
            {lat}, long {long}, offset_val {offset_val}',)
    else:
        return(output)

def handle_sleep(retry_request):
    reset_time=pd.to_datetime(retry_request.headers['ratelimit-resettime'])
    now=pd.Timestamp.utcnow()
    remaining_time=reset_time-now
    print(f'Should reset in {remaining_time}')
    sleeptime=remaining_time.seconds
    time.sleep(sleeptime)


def handle_multiple_queries(lat,long,li_of_offsets,output,snagflag):
    for offset_val in li_of_offsets:
        PARAMETERS=build_PARAMS(lat,long,1000,offset_val)
        return_request=requests.get(url = ENDPOINT, params= PARAMETERS, 
        headers = HEADERS)
        code=return_request.status_code
        if code==200:
            output=handle_good_return(return_request,output)
        else:
            print(f'Request Failed, {code}, going to retry in 10 seconds')
            try:
                output=handle_bad_response(lat,long,offset_val,output,snagflag)
            except:
                snagflag=True
                print('Hit a Snag')
                break
    return(output,snagflag)





12500

C:\Users\jacob\AppData\Local\Temp/ipykernel_26104/475346007.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.to_datetime(pd.datetime.now())


Timestamp('2022-10-25 15:28:14.323446')

In [58]:
(length-50)//50

14

In [62]:
list(range(50,length,50))

[50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750]

In [5]:
output = {}
output['businesses'] = []
last_remains=5000
try_num=0
snagflag=False
# Okay we're going to iterate over the geographic points
for _,row in data.iterrows():
    #backwards so 0 last
    try:
        lat=row[1]
        long=row[0]
        offset=0
        print(f'Starting query on ({lat},{long})')
        PARAMETERS=build_PARAMS(lat,long,1000,offset)
        starting_response = requests.get(url = ENDPOINT, params= PARAMETERS, headers = HEADERS)
        #Congrats it worked
        if starting_response.status_code==200:
            #write it out before doing anything else
            output=handle_good_return(starting_response,output)
            #Now what happens if we have more than 50 results
            length=starting_response.json()['total']
            #1000 is the limit so if its over we need to handle it differently
            # Easy Case
            if length>50 and length<=1000:
                # build out offset list
                li_of_offsets=list(range(50,length,50))
                #build output in seperate func
                output,snagflag=handle_multiple_queries(lat,long,li_of_offsets,output,snagflag)
                if snagflag:
                    raise Exception
            elif length>1000:
                #need to reset radius maybe?
                raise Exception
        else:
            output=handle_bad_response(lat,long,offset,output)
    except:
        going_out=output
    

    

Starting query on (33.219369763091024,-96.92274796186854)
Starting query on (33.219369763091024,-96.91376480902734)
Starting query on (33.219369763091024,-96.90478165618616)
Starting query on (33.219369763091024,-96.89579850334496)
Starting query on (33.219369763091024,-96.88681535050375)
Starting query on (33.219369763091024,-96.87783219766256)
Starting query on (33.219369763091024,-96.86884904482136)
Starting query on (33.219369763091024,-96.85986589198018)
Starting query on (33.219369763091024,-96.85088273913898)
Starting query on (33.219369763091024,-96.84189958629779)
Starting query on (33.219369763091024,-96.83291643345659)
Starting query on (33.219369763091024,-96.82393328061539)
Starting query on (33.219369763091024,-96.8149501277742)
Starting query on (33.219369763091024,-96.80596697493301)
Starting query on (33.219369763091024,-96.79698382209182)
Starting query on (33.219369763091024,-96.78800066925061)
Starting query on (33.219369763091024,-96.77901751640941)
Starting query 

In [23]:
pd.DataFrame(output['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,BbYl_rVZDFPQyT6tuNQDBw,up-inspired-kitchen-frisco,Up Inspired Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/eH4pAw...,False,https://www.yelp.com/biz/up-inspired-kitchen-f...,395,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 33.1219652678919, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '5285 Dallas Pkwy', 'address2': '...",+14695794197,(469) 579-4197,1228.302654
1,61iY8WwaOPWo2i5ewXpXLg,lazy-dog-restaurant-and-bar-plano,Lazy Dog Restaurant & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/qCvZ88...,False,https://www.yelp.com/biz/lazy-dog-restaurant-a...,1209,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 33.09276, 'longitude': -96.80659}","[delivery, pickup]",$$,"{'address1': '8401 Preston Rd', 'address2': No...",+14696091570,(469) 609-1570,4551.497179
2,ZadjbWA7kQi34IdSJfScow,kitchen-master-frisco,Kitchen Master,https://s3-media2.fl.yelpcdn.com/bphoto/qitrmd...,False,https://www.yelp.com/biz/kitchen-master-frisco...,279,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.0,"{'latitude': 33.15484908538643, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '9285 Preston Rd', 'address2': No...",+14693628001,(469) 362-8001,2874.194250
3,q1wWqj67JVn2BWYvTPtRDQ,the-alley-noodle-bar-mckinney-3,The Alley Noodle Bar,https://s3-media3.fl.yelpcdn.com/bphoto/4XDoEr...,False,https://www.yelp.com/biz/the-alley-noodle-bar-...,90,"[{'alias': 'noodles', 'title': 'Noodles'}, {'a...",4.5,"{'latitude': 33.16069, 'longitude': -96.7205}","[delivery, pickup]",$$,"{'address1': '7701 Stacy Rd', 'address2': 'Ste...",+14696251110,(469) 625-1110,9739.774764
4,_D_r2P-HYK2L-INqzqWzLQ,the-stix-icehouse-mckinney,The Stix Icehouse,https://s3-media2.fl.yelpcdn.com/bphoto/I--I96...,False,https://www.yelp.com/biz/the-stix-icehouse-mck...,178,"[{'alias': 'beergardens', 'title': 'Beer Garde...",4.5,"{'latitude': 33.178645, 'longitude': -96.622221}",[],$$,"{'address1': '301 W Eldorado Pkwy', 'address2'...",+12145928335,(214) 592-8335,19044.327682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,Sskny3NwoWK9LFEZvF_cBQ,oceanic-thai-kitchen-plano,Oceanic Thai Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/_oYP-k...,False,https://www.yelp.com/biz/oceanic-thai-kitchen-...,475,"[{'alias': 'thai', 'title': 'Thai'}]",4.5,"{'latitude': 33.019066, 'longitude': -96.795977}","[delivery, pickup]",$$,"{'address1': '1501 Preston Rd', 'address2': 'S...",+14697790749,(469) 779-0749,12777.386331
946,4FKHod0E7CqiepnxD6uVNQ,viet-spices-bep-co-nien-plano,Viet Spices Bep Co Nien,https://s3-media1.fl.yelpcdn.com/bphoto/lAkSpx...,False,https://www.yelp.com/biz/viet-spices-bep-co-ni...,5,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",4.0,"{'latitude': 33.0397833, 'longitude': -96.6970...","[delivery, pickup]",NaN,"{'address1': '3320 K Ave', 'address2': '', 'ad...",,,15335.284318
947,Lo9eM2etjIWBtu1GRjXHnw,salsa-tex-mex-prosper,Salsa Tex Mex,https://s3-media1.fl.yelpcdn.com/bphoto/9IXo8K...,False,https://www.yelp.com/biz/salsa-tex-mex-prosper...,139,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}]",4.0,"{'latitude': 33.248486238359455, 'longitude': ...",[delivery],$$,"{'address1': '1170 N Preston Rd', 'address2': ...",+19723464643,(972) 346-4643,13366.492348
948,FccMCWg7-EaSEN__d8CofQ,branding-iron-garland,Branding Iron,https://s3-media4.fl.yelpcdn.com/bphoto/B7UMR5...,False,https://www.yelp.com/biz/branding-iron-garland...,3,"[{'alias': 'restaurants', 'title': 'Restaurant...",3.5,"{'latitude': 32.968063, 'longitude': -96.620316}",[],NaN,"{'address1': '1535 E Brand Rd', 'address2': No...",+19722053958,(972) 205-3958,26015.865757


In [15]:
stash_last_response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [16]:
stash_last_response.json()['total']

3400

In [22]:
for i in range(0,100,50):
    print(i)

0
50


In [12]:
with open ('Frisco_pickle_test_data.pickle','wb') as handle:
    pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

+ Write output to text file

In [10]:
with open('business_search_enterprise.txt', 'w') as convert_file:
     convert_file.write(json.dumps(output))

In [4]:
# how many businesses?
count = 0
for x in output:
    if isinstance(output[x], list):
        count += len(output[x])
print(count)

1000


Get list of business ids to feed the endpoint for business details. One call = 1 business ID

In [11]:
my_list = output['businesses']
business_id = [d['id'] for d in my_list]

business_details = []

for i in business_id:
    b_id = i
    ENDPOINT = 'https://api.yelp.com/v3/businesses/%s' % b_id
    response = requests.get(url = ENDPOINT, headers = HEADERS)
    business_data = response.json()
    business_details.append(business_data)

Write file to txt

In [15]:
with open('business_details_enterprise.txt', 'w',encoding="utf-8") as fp:
    for item in business_details:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done
